#### Initial Setup

In [ ]:
#imported libraries
import pandas as pd
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import langdetect
import datetime
%matplotlib inline  
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import Imputer

import findspark; findspark.init()
import pyspark
import os
from pyspark.sql.types import *
import pyspark.sql.functions as func

os.environ["PYSPARK_SUBMIT_ARGS"] = (
  "--packages com.databricks:spark-csv_2.11:1.4.0 pyspark-shell"
)

try:
    sc = pyspark.SparkContext()
except Exception as e:
    print "SparkContext exists... Continuing on."
    
sqlCtx = pyspark.sql.SQLContext(sc)

#### Load files

In [ ]:
downloads = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_app_downloads.csv').drop('')
ratings = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_app_rating.csv').drop('')
usages = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_usage.csv').drop('')
revenues = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_revenue.csv').drop('')
output = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_final_downloads.csv').drop('')
prev_downloads = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_cumulative_downloads_2015-02.csv').drop('')
    
reviews = pd.read_csv('train_app_review.csv')
reviews_schema = StructType([
    StructField("id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("country",StringType(),True),
    StructField("rating",IntegerType(),True),
    StructField("date",StringType(),True),
    StructField("title",StringType(),True),
    StructField("version",StringType(),True),
    StructField("text",StringType(),True),
    StructField("reviewer",StringType(),True)
])
reviews = sqlCtx.createDataFrame(reviews,reviews_schema)

#### Generate Predictors

In [ ]:
old_dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y-%m-%d'))
dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y_%m_%d'))
for d in range(56):
    revenues = revenues.withColumnRenamed(old_dateRange[d],dateRange[d])
    usages = usages.withColumnRenamed(old_dateRange[d],dateRange[d])
    downloads = downloads.withColumnRenamed(old_dateRange[d],dateRange[d])

In [ ]:
predictors = downloads['id','name','category']

In [ ]:
# Generate the weekly downloads

sqlCtx.registerDataFrameAsTable(downloads, "downloads")

predictors = sqlCtx.sql("SELECT id, name, category, device , "+\
           "+".join(dateRange[0:7])+" AS week_1 ,"+\
           "+".join(dateRange[7:14])+" AS week_2 ,"+\
           "+".join(dateRange[14:21])+" AS week_3 ,"+\
           "+".join(dateRange[21:28])+" AS week_4 ,"+\
           "+".join(dateRange[28:35])+" AS week_5 ,"+\
           "+".join(dateRange[35:42])+" AS week_6 ,"+\
           "+".join(dateRange[42:49])+" AS week_7 ,"+\
           "+".join(dateRange[49:56])+" AS week_8 ,"+\
           "+".join(dateRange)+" AS download_sum \
           from downloads")

In [ ]:
# Make coefficients

def get_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    return  float(np.polyfit(range(56),np.cumsum(args[1:]),args[0])[0])
    
#Generate the step max and min 
def get_maxStep(maximum,*args):
    args=list(args)
    if (np.count_nonzero(args) == 0):
        return 0
    m = 0
    for d in range(1,56):
        if (args[d]!=0 and args[d-1]!=0):
            c = (args[d]-args[d-1])
            if (maximum and m < c):
                m = c
            if ( not maximum and m > c):
                m = c
    return m

def get_std(*args):
    return float(np.std(list(args)))

def get_nbMissing(*args):
    return list(args).count(-1)

sqlCtx.registerFunction("get_nbMissing", get_nbMissing,returnType=IntegerType())
sqlCtx.registerFunction("get_std", get_std,returnType=FloatType())
sqlCtx.registerFunction("get_maxStep", get_maxStep,returnType=IntegerType())
sqlCtx.registerFunction("get_coefficients", get_coefficients,returnType=FloatType())
temp_df = sqlCtx.sql("SELECT id,name,category, device \
, get_coefficients(0,"+",".join(dateRange)+") AS coef_0 \
,get_coefficients(1,"+",".join(dateRange)+") AS coef_1 \
,get_coefficients(2,"+",".join(dateRange)+") AS coef_2 \
,get_coefficients(3,"+",".join(dateRange)+") AS coef_3 \
,get_maxStep(True,"+",".join(dateRange)+") AS max_step \
,get_maxStep(False,"+",".join(dateRange)+") AS min_step \
,get_std("+",".join(dateRange)+") AS downloads_std \
,get_nbMissing("+",".join(dateRange)+") AS nb_missing \
 FROM downloads")


predictors = predictors.join(temp_df,(temp_df.id==predictors.id) & (temp_df.name==predictors.name) \
& (temp_df.category==predictors.category),"inner")



In [ ]:
predictors

In [ ]:
get_std([1,2,3,4])

In [ ]:
Row(**dict(row.asDict()))

In [ ]:
(lambda row: Row(**dict(row.asDict(), test=-1)))(row)

In [ ]:
downloads.head(1)

In [ ]:
predictors.head(1)